In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
# count the number of images in the respective classes 0 - Brain Tumor and 1 - Healthy
ROOT_DIR1 = "dataset"   # absolute location of dataset folder
number_of_images = {}
total_training_images=0


for dir in os.listdir(ROOT_DIR1):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR1,dir)))
  total_training_images+=number_of_images[dir]

print("Number of images in each folder of Training set:",number_of_images)
print("Total number of images in training set",total_training_images)

**We will split the data such that 70% for Train Data , 15% for Validation and rest 15% for Testing**

In [ ]:
# Function which will create a folder and spilts the images acoordingly in that percentage.

def dataFolder(path , split):

  ROOT = "directory\\"

  if not os.path.exists(ROOT+path):
    os.mkdir(ROOT+path)

    for dir in os.listdir(ROOT_DIR1):
      os.makedirs(ROOT+path+"\\"+dir)

      for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR1,dir)) , 
                                  size = (math.floor(split*number_of_images[dir])-5) , 
                                  replace=False):
        O = os.path.join(ROOT_DIR1,dir,img) # path and not the image
        D = os.path.join(ROOT+path,dir)
        shutil.copy(O,D)
        # os.remove(O)      # The image which wil copied in new directory(Train,Test,Validation) will be deleted from the original directory(here BrainTumorData)
  else:
    print(f"{path} Folder exists")

In [ ]:
# Making separate folders with certain number of images
dataFolder("Train",0.70)
dataFolder("Test",0.15)
dataFolder("Validation",0.15)

# **Model Build**

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator      
import keras


In [ ]:
# CNN  Model

model = Sequential()

model.add(Conv2D(filters=16 ,  kernel_size=(3,3) , activation='relu' , input_shape= (224,224,3)))   # Convolution layer ---for input layer input_shape . We can add padding='same' so that Conv2D shape will not reduce by 2

model.add(Conv2D(filters=36 ,  kernel_size=(3,3) , activation='relu' ))  # Convolution Layer
model.add(MaxPool2D(pool_size=(2,2)))     # Pooling Layer


model.add(Conv2D(filters=64 ,  kernel_size=(3,3) , activation='relu' ))  # Convolution Layer
model.add(MaxPool2D(pool_size=(2,2)))     # Pooling Layer


model.add(Conv2D(filters=128 ,  kernel_size=(3,3) , activation='relu' ))  # Convolution Layer
model.add(MaxPool2D(pool_size=(2,2)))     # Pooling Layer


model.add(Dropout(rate = 0.25))   # For Dropout--- so that model doesn't overfit----rate=25%

model.add(Flatten())  # For Flatten---doesn't take any parameter


model.add(Dense(units=64, activation= 'relu'))   # For Dense Layer

model.add(Dropout(rate=0.25))

model.add(Dense(units= 1 , activation= 'sigmoid'))  # Probability of being 1 or 0---output neuron will range from 0 to 1 and whichever value will be closer it belongs to that value

model.summary()


In [ ]:
model.compile(optimizer='adam' , loss= keras.losses.binary_crossentropy , metrics=['accuracy'])

### **Preprocessing or Preparing our data using Data Generator**

In [ ]:
# Creating a function for processing train images

def preprocessingImages1(path):
  """
  input : Path
  output : Generated Images
  """
  image_data = ImageDataGenerator(zoom_range= 0.2 , shear_range= 0.2 ,  rescale= 1/255 , horizontal_flip= True) # data augmentation
  image = image_data.flow_from_directory(directory = path, target_size = (224,224) , batch_size = 32 , class_mode= 'binary')  
  # taget_size should be same as input size i.e 224,224 and channel will be taken automatically
  # class_mode is binary because of binary classification

  return image



In [ ]:
path = "directory\\Train"
train_data = preprocessingImages1(path)


In [ ]:
# Creating a function for processing testing and validation images

def preprocessingImages2(path):
  """
  input : Path
  output : Generated Images
  """
  image_data = ImageDataGenerator(rescale= 1/255)
  image = image_data.flow_from_directory(directory = path, target_size = (224,224) , batch_size = 32 , class_mode= 'binary')  
  # taget_size should be same as input size i.e 224,224 and channel will be taken automatically
  # class_mode is binary because of binary classification

  return image



In [ ]:
path = "directory\\Test"
test_data = preprocessingImages2(path)


In [ ]:
path = "directory\\Validation"
val_data = preprocessingImages2(path)

In [ ]:
# Early stopping and model check point
# early stopping will stop the model execution if it thinks that it is trained enough in that time frame

from keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping
es = EarlyStopping(monitor="val_accuracy" , min_delta=0.01 , patience= 3 , verbose= 1 , mode= 'auto')

# Model checkpoint
mcp = ModelCheckpoint(monitor="val_accuracy" , filepath="bestmodel.h5" ,  verbose= 1 , save_best_only= True, mode= 'auto')

cb = [es,mcp]   # array  callbacks


# **Model Training**

In [ ]:
# history
hs = model.fit_generator(generator= train_data ,
                         steps_per_epoch= 8 , 
                         epochs= 300 , 
                         verbose= 1 , 
                         validation_data= val_data , 
                         validation_steps= 6 ,
                         callbacks= cb)    


In [ ]:
# Model Graphical Representation
h = hs.history
h.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')

plt.title("Accuracy Vs Validation_Accuracy")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h['loss'])
plt.plot(h['val_loss'], c='red')

plt.title("Loss Vs Validation_Loss")
plt.show()

# **Model Accuracy**

In [ ]:
# Model Accuracy
from keras.models import load_model
model = load_model("bestmodel.h5")


In [ ]:
acc = model.evaluate_generator(test_data)[1]
print(f"The accuracy of our model is {acc*100} %")

In [ ]:
# !pip install Keras-Preprocessing
import math
from keras_preprocessing.image import load_img, img_to_array

In [ ]:
# giving that image path which we haven't preprocessed i.e from dataset folder

path = "dataset\\healthy\\6.jpg"


img = load_img(path ,  target_size= (224,224))
input_arr = img_to_array(img)/255   


plt.imshow(input_arr)
plt.show()



input_arr.shape 

prediction_classes = (model.predict(input_arr) > 0.5).astype("int32")[0][0]



if prediction_classes==0:
  print("The Brain MRI has no tumor")
else:
  print("The Brain MRI is having a tumor")